In [1]:
%%capture
!pip install rdkit
!pip install torch_geometric

In [2]:
import pandas as pd
import torch
from torch_geometric.utils import from_smiles


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.load('/content/model_lipo_tuned.pth', map_location=device)
model.eval()

AttentiveFP(in_channels=9, hidden_channels=64, out_channels=1, edge_dim=3, num_layers=6, num_timesteps=2)

In [6]:
def predict_lipophilicity(smiles_list):
    predictions = []
    for smile in smiles_list:
        # Convert SMILES to graph
        g = from_smiles(smile)
        g.x = g.x.float()
        g = g.to(device)

        # Add batch information
        g.batch = torch.tensor([0] * g.num_nodes, dtype=torch.long, device=device)

        # Perform prediction
        with torch.no_grad():
            pred = model(g.x, g.edge_index, g.edge_attr, g.batch)
        predictions.append(pred.item())
    return predictions

In [10]:
external_smiles = [
    'OCC1=C(O)C=C(O)C=C1',  # Replace with actual SMILES strings you want to predict
    'C12=CC=CC=C1C=C3C(C=C(C=CC=C4)C4=C3)=C2'
]

lipophilicity_predictions = predict_lipophilicity(external_smiles)

for smile, pred in zip(external_smiles, lipophilicity_predictions):
    print(f'SMILES: {smile}, Predicted Lipophilicity: {pred:.4f}')

SMILES: OCC1=C(O)C=C(O)C=C1, Predicted Lipophilicity: 0.2754
SMILES: C12=CC=CC=C1C=C3C(C=C(C=CC=C4)C4=C3)=C2, Predicted Lipophilicity: 4.1201


In [8]:
lipophilicity_predictions

[0.275351345539093, 4.120083332061768]